In [74]:
#LIBS
import pandas as pd
import urllib.request
import json
import pymysql
from pandas.io import sql
from sqlalchemy import create_engine
import numpy as np

#DB DETAILS
host="computacaonuvem.cgbnndsgdpug.us-east-2.rds.amazonaws.com"
port=3306
dbname="computacaonuvem"
user="cnroot"
password="#CN2019!"

#CONNECT DO DB
db = pymysql.connect(
    host,
    user=user,
    port=port,
    passwd=password,
    db=dbname
)

In [ ]:
#Forces
forces = pd.read_json(urllib.request.urlopen("https://data.police.uk/api/forces").read())

#Forces  and Engagement Methods
forces_details = pd.DataFrame({
    'id':[],
    'name':[],
    'description':[],
    'telephone':[],
    'url':[],
})

forces_egagement_methods = pd.DataFrame({
    'force_id':[],
    'url':[],
    'description':[],
    'title':[]
})

for force_id in forces.id:
    
    #Get force details (based on id)
    response = pd.read_json(urllib.request.urlopen("https://data.police.uk/api/forces/"+force_id).read())
    
    print(response[:1].values[0][3])
    
    #Force details
    forces_details = forces_details.append({
            'id': force_id,
            'name': response[:1].values[0][3],
            'description': response[:1].values[0][0],
            'telephone': response[:1].values[0][4],
            'url': response[:1].values[0][5]
    },ignore_index=True)
    
    #Force engagement methods
    for n_index, engagement_details in enumerate([row[1][1] for row in response.iterrows()]):
        forces_egagement_methods = forces_egagement_methods.append({
            'force_id': force_id,
            'url': engagement_details['url'],
            'description': engagement_details['description'],
            'title': engagement_details['title']
        },ignore_index=True)
        
#Add IDs to forces_egagement_methods
forces_egagement_methods = forces_egagement_methods.reset_index() #Generate ids
forces_egagement_methods.columns = ['id', 'description', 'force_id', 'title', 'url'] #Rename columns
forces_egagement_methods['id'] = forces_egagement_methods['id'] + 1 #Make it so that id starts at 1


#Officers
officers = pd.DataFrame({
    'name':[],
    'rank':[],
    'bio':[],
    'force_id':[]
})

for force_id in forces.id:
    
    #Get officers for force
    officers_response = pd.read_json(urllib.request.urlopen("https://data.police.uk/api/forces/"+force_id+"/people").read())
    
    #Track progress on forces
    print(force_id)
    
    #Parse offiers
    for officer in officers_response.iterrows(): 
        
        officers = officers.append({
            'name':officer[1]['name'],
            'rank':officer[1]['rank'],
            'bio':officer[1]['bio'],
            'force_id':force_id
        },ignore_index=True)

#Add IDs to officers
officers = officers.reset_index() #Generate id for officers
officers.columns = ['id', 'bio', 'force_id', 'name', 'rank'] #Rename columns
officers['id'] = officers['id'] + 1 #Make it so that id starts at 1

In [ ]:
#Export CSVs
forces_details[['id', 'description', 'name', 'telephone', 'url']].to_csv('forces.csv', index=False)
forces_egagement_methods[['id', 'title', 'description', 'url', 'force_id']].to_csv('forces_engagement_methods.csv', index=False)
officers[['id', 'name', 'rank', 'bio', 'force_id']].to_csv('officers.csv', index=False)  